# CIFAR-10 Using Pytorch

### Test for CUDA 

In [ ]:
import torch
import numpy as np

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print("CUDA is not present, training on CPU...")
else:
    print("Training on GPU.....")
    

---
### Loading Data And Data Augmentation 




In [1]:
from torchvision import datasets
import torchvision.transfroms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

#number of subprocesses to use for data loading
num_workers = 0
#how many examples per batch
batch_size = 32
#percentage of training set to use as validation set
valid_size = 0.2

#Converting data into a normalized torch.FloatTensor
transforms = transforms.Compose([
    transforms.Randomrotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

# Chose Training and Test Data
train_data = datasets.CIFAR10('data', train=True, download=True, transform=transforms)
test_data = datasets.CIFAR10('data', train=False, download=True, transform=transforms)

# obtaining validation indices
indices = list(range(len(train_data)))
np.random.shuffle(indices)
split = int(np.floor(valid_size * len(train_data)))
train_idx, valid_idx = indices[split: ], indices[:split]

# define sampler for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
validation_sampler = SubsetRandomSampler(validation_idx)

# prepare data loaders
train_loader = torch.utils.data.Dataloader(train_data, batch_size=batch_size,
                                           sampler=train_sampler, num_workers= num_workers)
valid_loader = torch.utils.data.Dataloader(train_data, batch_size=batch_size,
                                           sampler=validation_sampler, num_workers=num_workers)
test_loader = torch.utils.data.Dataloader(test_data, batch_size, num_workers=num_workers)

# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

ModuleNotFoundError: No module named 'torchvision'

---
### Network Architecture 

In [2]:
### import torch.nn as nn
import torch.nn.functional as F

# CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolution layer (sees 32x32x3 image tensor)
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        # convolution layer (sees 16x16x16 tensor)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        # convolution layer (sees 8x8x32 tensor)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # maxpooling layer
        self.pool = nn.MaxPool2d(2,2)
        # linear Layer (64*4*4 -> 500)
        self.fc1 = nn.Linear(64*4*4, 500)
        # Linear Layer (500 -> 10)
        self.fc2 = nn.Linear(500,10)
        # Dropout layer (p=0.2)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        # Add a sequence of convolution and maxpooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # Flatten the image input
        x = x.view(-1, 64*4*4)
        # Dropout
        x = self.dropout(x)
        # add 1st Fully Connecter layer, with relu activation
        x = F.relu(self.fc1(x))
        # add dropout
        x = self.dropout(x)
        # add 2nd fc layer, with relu activation function
        x = self.fc2(x)
        return(x)
    
    # create a complete CNN
    model = Net()
    print(model)
    
    # move tensor to GPU if CUDA is available
    if train_on_gpu:
        model.cuda()
        

ModuleNotFoundError: No module named 'torch'

---
### Optimizer And Loss Function 

In [ ]:
import torch.optim as optim

# loss function
loss_fn = nn.CrossEntropy()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

---
### Train the network 

In [3]:
# number of epochs to train the model
n_epochs = 20

valid_loss_min = np.Inf

for epoch in range(1, n_epochs+1):
    train_loss = 0.0
    valid_loss = 0.0
    
    ##################
    # train the model#
    ##################
    
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # move tensors to GPU if CUDA available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass
        output = model(data)
        # calculate the batch loss
        loss = loss_fn(output, target)
        # backward pass
        loss.backward()
        # perform a single step/ weight-update
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################
    # validate the model #
    ######################
    
    model.eval()
    for batch_idx, (data, target) in enumerate(validation_loader):
        # move tensors to GPU if CUDA available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass
        output = model(data)
        # calculate the batch loss
        loss = loss_fn(output, target)
        # update average validation loss
        valid_loss += loss.item()*data.size(0)
        
    # calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(validation_loader.sampler)
    
    # print training/validation statistics
    print("Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}".format(
          epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} ======> {:./6f}). Saving Model...'.format(
              valid_loss_min,
              valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

NameError: name 'n' is not defined